[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/choROPeNt/gzlb/blob/main/notebooks/GZLB-UE6.3.ipynb
)

# GZLB
## UE6: Anisotropes Werkstoffverhalten, Verallgemeinertes Hooke'sches Gesetz, Festigkeitshypothesen



### Aufgabe 6.3: Versagenskriterium II

geg: Materialdaten und Randbedingungen

| elastische Eigenschaften   |      Festigkeiten      | 
|----------|:-------------:|
| $E_\parallel = 150 \,\text{GPa}$ |  $R_\parallel^+=1500\,\text{MPa}$ |
| $E_\bot = 8 \,\text{GPa}$  |    $R_\parallel^-=1200\,\text{MPa}$  |
| $G_{\bot\parallel }= 4,5 \, \text{GPa}$| $R_\bot^+=50\,\text{MPa}$ |
| $\nu_{\bot\parallel} = 0,28$| $R_\bot^-=110\,\text{MPa}$ |
| | $R_{\parallel\bot}=70\,\text{MPa}$ |
| TsaiWu Interaktionskoeffizient | $F_{12} = 0 $  |



ges: 
- Tangential und Axialspannungen ($\sigma_\varphi,\sigma_z$) gemäß der Kesselformel
- Tangential und Axialdehungen ($\varepsilon_\varphi,\varepsilon_z$) 
- Reservefaktor $f_{res}$ nach dem Versagenskriterium von TsaiWu

#### Schritt 1
importieren des Packages Numpy

In [ ]:
import numpy as np

#### Eingabe der Parameter


In [ ]:
E_11 = 150E3 ## MPa
E_22 = 8E3 ## MPa
G_12 = 4.5E3 ## MPa

nu_12 = 0.28 ## MPa


R11t = 1500 ## MPa
R11c = 1200 ## MPa
R22t = 50 ## MPa
R22c = 110 ## MPa
R12 = 70 ## MPa

D = 2e3   ## mm
h = 10   ## mm
p = 10 ## bar

### Berechnung des Spannungstensors mittles Kesselformel

$$\begin{pmatrix} \sigma_\varphi \\ \sigma_z \\ \tau_{\varphi z}\end{pmatrix} \approx \frac{pD}{4h} \begin{pmatrix}2 \\ 1\\ 0 \end{pmatrix}$$

In [ ]:
stress = p/10*D/(4*h) * np.array([2,1,0])

print("Tangentialspannung simga_phi = %.1f\nAxialspannung simga_z = %.1f\nSchubspannung tau_phiz = %.1f" %(*stress,))

## Berechnung der Spanungen und Dehnungen 
### Erweitertes Hook'sche Gesetz

$$ \begin{pmatrix} \varepsilon_1 \\ \varepsilon_2 \\ \gamma_{12} \end{pmatrix} = \begin{bmatrix} S_{11} & S_{12} & 0\\
S_{21} & S_{22} & 0\\
0 & 0 & S_{66} \end{bmatrix} \begin{pmatrix} \sigma_1 \\ \sigma_2 \\ \tau_{12}\end{pmatrix} $$

dabei ist die Nachgiebigkeitsmatrix für transersalisotrope Werkstoffe folgt definiert:

$$[S_{ij}]_{(\parallel,\bot)} = \begin{bmatrix} S_{11} & S_{12} & S_{16}\\
S_{21} & S_{22} & S_{26}\\
S_{61} & S_{62} & S_{66} \end{bmatrix}=\begin{bmatrix} \frac{1}{E_{\parallel}} & -\frac{\nu_{\parallel\bot}}{E_\parallel} & 0\\
-\frac{\nu_{\parallel\bot}}{E_\parallel} & \frac{1}{E_{\bot}} & 0\\
0 & 0 & \frac{1}{G_{\parallel\bot}} \end{bmatrix}$$





In [ ]:
S11 =   1/E_11
S22 =   1/E_22  
S66 =   1/G_12
S12 =   -nu_12/E_11
S21 =   S12
S16 =   0
S26 =   0
S61 =   S16 # symmetry
S62 =   S26 # symmetry

In [ ]:
S = np.array([[S11,S12,S16],[S21,S22,S26],[S61,S62,S66]])

In [ ]:
strain = np.matmul(S,stress)
print("Tangentialdehnung epsilon_phi = %.1E\nAxialdehung epsilon_z = %.1E\nSchubdehnung gamma_phiz = %.1E" %(*strain,))

### Berechnung des Versagenskriteriums nach TsaiWu

Die allgeimeine Darstellung des TsaiWu-Versagenskriterium für den ebenen Spannungszustand ist wie folgt:
$$ \sum_{i,j=1,2,6} F_{i}\sigma_i + F_{ij} \sigma_{i}\sigma_{j}\leq 1$$
Dabei sind die Spannungen $\sigma_i$ für das $(\parallel,\bot)$-(Material)-Koordinatensystem wie folgt definiert (Voigt-Notation; pseudo-vekotrielle Darstellung):

$$\underline{\sigma}_{(\parallel,\bot)}= \left[\sigma_\parallel, \sigma_\bot, \tau_{\parallel,\bot} \right]=\left[\sigma_1, \sigma_2, \sigma_6 \right]$$

Daraus folgt die folgende Definition des TsaiWu Versagenskriteriums
$$F_{1} \sigma_\parallel + F_{11} \sigma_\parallel^2 +F_{12}\sigma_{\parallel\bot} F_{2} \sigma_\bot + F_{2} \sigma_\bot^2 + F_{66}\tau_{\parallel\bot}\leq 1$$

Hinweis: 

- lineare Schubspannungsglieder sind per Definition zu vernachlässigen
- $F_{12}=0$ (konservative Auslegung)




#### Berechnung des Festigkeitskoefizienten $F_{i}$ und $F_{ij}$

$$F_{1} = \frac{1}{R_1^+}- \frac{1}{R_1^-} \qquad F_{2} = \frac{1}{R_2^+}- \frac{1}{R_2^-}$$

$$F_{11} = \frac{1}{R_1^+R_1^-}\qquad F_{22} = \frac{1}{R_2^+R_2^-}\qquad F_{66} = \frac{1}{\left(R_{12}\right)^2}$$



In [ ]:
F1 = 1/R11t-1/R11c
F2 = 1/R22t-1/R22c

F11 =   1/(R11t*R11c)
F22 =   1/(R22t*R22c)
F66 =   1/(R12)**2
F12 =   0

In [ ]:
print("F1: %.3E; F2: %.3E;F11: %.3E; F22: %.3E; F66: %.3E; F12: %.3E" %(F1,F2,F11,F22,F66,F12))

In [ ]:
def tsaiwu2D(stress,coeff):
    """
    input:  
        -   stress: 3x1 numpy array with stress components 
        -   coeff: 6x1 numpy array with the strength coeff F1,F2,F11,F22,F66,F12
    output: 
        -   value: value of TsaiWu criteria
        -   fres: reverse faktor of TsaiWu criteria
    """
    F1 = coeff[0]
    F2 = coeff[1]
    F11 =   coeff[2]
    F22 =   coeff[3]
    F66 =   coeff[4]
    F12 =   coeff[5]

    # calcualte the value of TsaiWu criteria
    value = F1*stress[0]+ F11*stress[0]**2 +F2*stress[1]+ F22*stress[1]**2 + F66*stress[2]**2

    # calcualte the reserve factor of TsaiWu criteria
    a = F11*stress[0]**2 + F12*stress[0]*stress[1] + F22*stress[1]**2+F66*stress[2]**2
    b = F1*stress[0] +F2*stress[1]
    c = -1
    # function np.roots return the roots of a polynomial with coefficients given in p
    # https://numpy.org/doc/stable/reference/generated/numpy.roots.html
    fres = np.max(np.roots([a, b, c]) )
    
    # return all values
    return value, fres

In [ ]:
value, fres = tsaiwu2D(stress,[F1,F2,F11,F22,F66,F12])

print("The value of TsaiWu cirteria: %.3E\nand the reserve factor fres: %.3E" %(value,fres))